In [ ]:
import gym
import gym_pygame
import torch
import gym_mp
import numpy as np
import time

import ray
ray.init(num_gpus=1)



In [7]:
def gen():
    for i in range(10):
        yield i, i*10
        
for (i,j) in gen():
    print(i,j)
    
def bar():
    return 1,2

(a,b) = bar()

def foo((a,b)): #syntax error
    print(a,b)
    
foo(1,2)

    

0 0
1 10
2 20
3 30
4 40
5 50
6 60
7 70
8 80
9 90
1 2


In [ ]:
import pyworld.toolkit.tools.visutils.jupyter as J


env_f = lambda : gym_pygame.envs.Expander()


lit = gym_mp.mp_episodes(env_f, workers=3, 
                          mode=gym_mp.mode.sa, n=1, 
                          max_length=1000)
#lit = lit.for_each(lambda x: [torch.from_numpy(y) for y in x])
for x in lit.gather_async():
    s, a = x
    print(s.shape)
    J.images(s)
    print(s[0][0], a.shape)
    



In [ ]:
import os

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
    
use_gpu.remote()

In [ ]:
import os
print(ray.get_gpu_ids())
#os.environ["CUDA_VISIBLE_DEVICES"]

In [ ]:
gym_mp.iterators.episode('CartPole-v0', mode=gym_mp.mode.sa)

In [ ]:

lit = gym_mp.mp_episodes('Pong-v0', workers=2, 
                            mode=gym_mp.mode.sa, n=1, 
                            max_length=1000)
lit = lit.for_each(lambda x: [torch.from_numpy(y).cuda() for y in x]) #hmm... it seems complicated to enable cuda support -- might require a custom ParallelIteratorWorker

for x in lit.gather_async():
    s, a = x
    s = s.cuda()
    print(s[0][0], a.shape)
    


In [ ]:
class TestEnv(gym.Env):
    
    def __init__(self):
        super(TestEnv, self).__init__()
        self.name = "env"
        self.i = 0
        self.n = 3
        self.action_space = gym.spaces.Discrete(2)
    
    def step(self, action):
        self.i += 1
        time.sleep(np.random.randint(1000)/1000)
        done = self.i > self.n
        return "{0}-{1}-{2}".format(self.name, self.i, action), 0., done

    def reset(self):
        self.i = 0
        return "{0}-{1}-{2}".format(self.name, self.i, -1)

In [ ]:

policy_f = lambda : gym_mp.policy.uniform(gym.spaces.Discrete(2))

env_f = TestEnv
lit = gym_mp.async_iterator(env_f, policy_f, repeat=True)

for s in lit:
    print(*s)


In [ ]:
import time
import ray.util.iter

class wait_iter:
    
    def __init__(self, n):
        self.n = n
        self.i = 0
    
    def __next__(self):
        time.sleep(1)
        self.i += 1
        if self.i > self.n:
            raise StopIteration()
        return self.i
    
    def __iter__(self):
        return self
    
def gen(n):
    for i in range(n):
        time.sleep(1)
        yield i

class wait_iter2:
    
    def __init__(self, n):
        self.n = n
    
    def __iter__(self):
        return gen(self.n)
        
    
it = ray.util.iter.from_iterators([range(10), wait_iter2(10)])

lit = it.gather_async()

for i in lit:
    print(i)